#  Implementation of BB84 Protocol:

In [1]:
from random import getrandbits # Importing necessary libraries
import qiskit as q

In [2]:
def select_encoding(length):   # Function for creating random bits by sender for 
    sender_bitstring = ""      # sender bit string and sender bases.
    sender_bases = ""
    for i in range(length):
        sender_bitstring += (str(getrandbits(1)))
        sender_bases += (str(getrandbits(1)))
    return sender_bitstring, sender_bases

In [3]:
def select_measurement(length):
    receiver_bases = ""      # Creating receiver bases randomly
    for i in range(length):
        receiver_bases += (str(getrandbits(1)))
    return receiver_bases

In [4]:
def encode(sender_bitstring, sender_bases):  # Encoder function to
    encoded_qubits = []                    # be used by sender
    for i in range(len(sender_bitstring)):
        qc = q.QuantumCircuit(1,1)
        if sender_bases[i] == "0":
            if sender_bitstring[i] == "0":
                pass
            elif sender_bitstring[i] == "1":
                qc.x(0)
        elif sender_bases[i] == "1":
            if sender_bitstring[i] == "0":
                qc.h(0)
            elif sender_bitstring[i] == "1":
                qc.x(0)
                qc.h(0)
        encoded_qubits.append(qc)
    return encoded_qubits

In [5]:
def measure(receiver_bases, encoded_qubits, backend): # Measuring encoded
    receiver_bitstring = ''                           # bits by receiver 
    for i in range(len(encoded_qubits)):  # using his/her produced bases   
        qc = encoded_qubits[i]
        if receiver_bases[i] == "0":
            qc.measure(0,0)
        elif receiver_bases[i] == "1":
            qc.h(0)
            qc.measure(0,0)
        job = q.execute(qc, backend=backend, shots = 1) 
        results = job.result()
        counts = results.get_counts()
        measured_bit = max(counts, key=counts.get)
        receiver_bitstring += measured_bit 
    return receiver_bitstring

In [6]:
def intercept_of_eve(encoded_qubits): # Creating situation if eve intercepts
    eve_bases= select_measurement(len(encoded_qubits))
    eve_measure_encoded_qubits= measure(eve_bases, encoded_qubits, q.Aer.get_backend('qasm_simulator'))
    eve_new_bases, eve_new_bitstring= select_encoding(len(encoded_qubits))
    eve_create_new_encoded_qubits= encode(eve_new_bases, eve_new_bitstring)
    QUANTUM_PATH= eve_create_new_encoded_qubits
    return QUANTUM_PATH

In [7]:
def receiver_compare_bases(sender_bases, receiver_bases):
    indices = []
    for i in range(len(sender_bases)):
        if sender_bases[i] == receiver_bases[i]:
            indices.append(i)
    return indices

In [8]:
def construct_key_from_indices(bitstring, indices): # Function to make keys
    key = ''
    for idx in indices:
        key = key + bitstring[idx] 
    return key

In [9]:
KEY_LENGTH = 1000     # An implementated example with/ without eve.
QUANTUM_PATH = []
CLASSICAL_PATH = []

In [10]:
sender_bitstring, sender_bases = select_encoding(KEY_LENGTH)

In [11]:
receiver_bases = select_measurement(KEY_LENGTH)

In [12]:
encoded_qubits = encode(sender_bitstring, sender_bases)

In [13]:
QUANTUM_PATH = encoded_qubits

In [14]:
random_situation= str(getrandbits(1))
# print(random_situation)          # Creating random situation 
if random_situation== '0':       # Will eve intercept or not ?!
    QUANTUM_PATH= intercept_of_eve(QUANTUM_PATH)
elif random_situation== '1':
    pass

In [16]:
sim_backend = q.Aer.get_backend('qasm_simulator') # Running in simulator
receiver_bitstring = measure(receiver_bases, QUANTUM_PATH, sim_backend)

In [17]:
CLASSICAL_PATH = sender_bases

In [18]:
agreeing_bases = receiver_compare_bases(CLASSICAL_PATH, receiver_bases)

In [19]:
CLASSICAL_PATH = agreeing_bases

In [20]:
sender_key = construct_key_from_indices(sender_bitstring, CLASSICAL_PATH)
receiver_key = construct_key_from_indices(receiver_bitstring, agreeing_bases)
print("sender_key: ", sender_key[:50])
print("receiver_key: ", receiver_key[:50])
print("sender's key is equal to receiver's key: ", sender_key == receiver_key)
if sender_key != receiver_key:
    print('The path is not secure!')
elif sender_key == receiver_key:
    print('The path is secured!')

sender_key:  01010100000101011001110110011101010101011000011101
receiver_key:  11000011101011111110111100101011101001111100011000
sender's key is equal to receiver's key:  False
The path is not secure!
